In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Dropout, GlobalMaxPool1D, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, TimeDistributed, Attention
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [3]:
# Load datasets
mbti_dataset = pd.read_csv("mbti_1.csv", encoding='ISO-8859-1')
big5_dataset = pd.read_csv("big5_1.csv", encoding='ISO-8859-1')

In [4]:
# Rename and select relevant columns
mbti_dataset.rename(columns={'type': 'MBTI', 'posts': 'TEXT'}, inplace=True)
big5_columns_to_keep = ['TEXT', 'cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
big5_dataset = big5_dataset[big5_columns_to_keep]

In [5]:
# Function to convert Big5 traits to MBTI type
def big5_to_mbti(row):
    mbti = ''
    mbti += 'E' if row['cEXT'] == 'y' else 'I'
    mbti += 'N' if row['cOPN'] == 'y' else 'S'
    mbti += 'F' if row['cAGR'] == 'y' else 'T'
    mbti += 'J' if row['cCON'] == 'y' else 'P'
    return mbti

In [6]:
# Apply conversion
big5_dataset['MBTI'] = big5_dataset.apply(big5_to_mbti, axis=1)
full_dataset = pd.concat([mbti_dataset[['TEXT', 'MBTI']], big5_dataset[['TEXT', 'MBTI']]], ignore_index=True)


In [7]:
# Preprocess text data
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
import re

def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(filtered_words)

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
full_dataset['TEXT'] = full_dataset['TEXT'].apply(preprocess_text)

In [11]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(full_dataset['TEXT'], full_dataset['MBTI'], test_size=0.2, random_state=42)


In [27]:
# Tokenize text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=100, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=100, padding='post')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = np.max(y_train_encoded) + 1
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Checkpoint callback
checkpoint_path = "/content/drive/MyDrive/Model_data/model_checkpoint_personality_pred.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


In [15]:
# Build the model
embedding_dim = 64
vocab_size = len(tokenizer.word_index) + 1

In [16]:
from keras.layers import GlobalAveragePooling1D
input_text = Input(shape=(100,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=100)(input_text)
bi_lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = Attention()([bi_lstm, bi_lstm])
pooling_layer = GlobalAveragePooling1D()(attention)  # This collapses the sequence dimension
dense_layer = Dense(64, activation='relu')(pooling_layer)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

In [17]:
model = Model(inputs=input_text, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train_pad, y_train_cat, epochs=50, batch_size=64, validation_data=(X_test_pad, y_test_cat))


In [40]:
from keras.layers import Layer
import keras.backend as K

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(name='att_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(name='att_bias',
                                 shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        e = K.tanh(K.dot(inputs, self.W) + self.b)
        e = K.squeeze(e, axis=-1)
        alpha = K.softmax(e)
        context = inputs * K.expand_dims(alpha, -1)
        context = K.sum(context, axis=1)
        return context

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Use this attention in your model
attention = AttentionLayer()(bi_lstm)


In [41]:
from keras.layers import GlobalAveragePooling1D
input_text = Input(shape=(100,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=100)(input_text)
bi_lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = Attention()([bi_lstm, bi_lstm])
pooling_layer = GlobalAveragePooling1D()(attention)  # This collapses the sequence dimension
dense_layer = Dense(64, activation='relu')(pooling_layer)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

In [42]:
model = Model(inputs=input_text, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_pad, y_train_cat, epochs=50, batch_size=64, validation_data=(X_test_pad, y_test_cat))


Epoch 1/50
140/140 [==============================] - 20s 100ms/step - loss: 2.4706 - accuracy: 0.1764 - val_loss: 2.3575 - val_accuracy: 0.1907
Epoch 2/50
140/140 [==============================] - 8s 56ms/step - loss: 2.3163 - accuracy: 0.1917 - val_loss: 2.3857 - val_accuracy: 0.1768
Epoch 3/50
140/140 [==============================] - 5s 32ms/step - loss: 2.1659 - accuracy: 0.2294 - val_loss: 2.3332 - val_accuracy: 0.2104
Epoch 4/50
140/140 [==============================] - 4s 28ms/step - loss: 1.9861 - accuracy: 0.2862 - val_loss: 2.4135 - val_accuracy: 0.2279
Epoch 5/50
140/140 [==============================] - 4s 31ms/step - loss: 1.7836 - accuracy: 0.3520 - val_loss: 2.5760 - val_accuracy: 0.2373
Epoch 6/50
140/140 [==============================] - 3s 21ms/step - loss: 1.5864 - accuracy: 0.4227 - val_loss: 2.7730 - val_accuracy: 0.2176
Epoch 7/50
140/140 [==============================] - 3s 21ms/step - loss: 1.4193 - accuracy: 0.4925 - val_loss: 3.0739 - val_accuracy: 0.22

In [43]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.regularizers import l2

In [44]:
# Build the model
embedding_dim = 64
vocab_size = len(tokenizer.word_index) + 1
input_text = Input(shape=(100,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=100)(input_text)
bi_lstm = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01)))(embedding_layer)
dropout = Dropout(0.5)(bi_lstm)
attention = Attention()([dropout, dropout])
dropout_2 = Dropout(0.5)(attention)
dense_layer = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(dropout_2)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

In [45]:
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Attention

# Parameters
embedding_dim = 64
vocab_size = len(tokenizer.word_index) + 1  # Make sure tokenizer is defined above
num_classes = 16  # Change this to the actual number of classes you have

# Model architecture
input_text = Input(shape=(100,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=100)(input_text)
bi_lstm = Bidirectional(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.01)))(embedding_layer)  # return_sequences is now False
dropout = Dropout(0.5)(bi_lstm)
dense_layer = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(dropout)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

# Create and compile the model
model = Model(inputs=input_text, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train_pad, y_train_cat, epochs=50, batch_size=64, validation_data=(X_test_pad, y_test_cat))


Epoch 1/50
140/140 [==============================] - 19s 98ms/step - loss: 3.4682 - accuracy: 0.1710 - val_loss: 2.6515 - val_accuracy: 0.1310
Epoch 2/50
140/140 [==============================] - 7s 52ms/step - loss: 2.5540 - accuracy: 0.1713 - val_loss: 2.4780 - val_accuracy: 0.1812
Epoch 3/50
140/140 [==============================] - 4s 30ms/step - loss: 2.4505 - accuracy: 0.1741 - val_loss: 2.4164 - val_accuracy: 0.1929
Epoch 4/50
140/140 [==============================] - 4s 26ms/step - loss: 2.4073 - accuracy: 0.1861 - val_loss: 2.4016 - val_accuracy: 0.1925
Epoch 5/50
140/140 [==============================] - 3s 25ms/step - loss: 2.3831 - accuracy: 0.1879 - val_loss: 2.4183 - val_accuracy: 0.1889
Epoch 6/50
140/140 [==============================] - 4s 27ms/step - loss: 2.3654 - accuracy: 0.1936 - val_loss: 2.4120 - val_accuracy: 0.1866
Epoch 7/50
140/140 [==============================] - 3s 23ms/step - loss: 2.3496 - accuracy: 0.1934 - val_loss: 2.4069 - val_accuracy: 0.186

In [46]:
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Attention
from tensorflow.keras.regularizers import l2

# Parameters
embedding_dim = 64
vocab_size = len(tokenizer.word_index) + 1  # Ensure tokenizer is defined previously
num_classes = 16  # Set this to the number of your output classes
l2_reg = 0.01  # Regularization factor

# Model architecture
input_text = Input(shape=(100,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=100)(input_text)
bi_lstm = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(l2_reg)))(embedding_layer)
dropout1 = Dropout(0.5)(bi_lstm)
attention = Attention()([dropout1, dropout1])
pooling_layer = GlobalAveragePooling1D()(attention)
dropout2 = Dropout(0.5)(pooling_layer)
dense_layer = Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(dropout2)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

# Create and compile the model
model = Model(inputs=input_text, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
# Train the model
model.fit(X_train_pad, y_train_cat, epochs=50, batch_size=64, validation_data=(X_test_pad, y_test_cat))


Epoch 1/50
140/140 [==============================] - 20s 100ms/step - loss: 3.5020 - accuracy: 0.1650 - val_loss: 2.6578 - val_accuracy: 0.1853
Epoch 2/50
140/140 [==============================] - 6s 46ms/step - loss: 2.5448 - accuracy: 0.1701 - val_loss: 2.5031 - val_accuracy: 0.1853
Epoch 3/50
140/140 [==============================] - 6s 41ms/step - loss: 2.4673 - accuracy: 0.1707 - val_loss: 2.4303 - val_accuracy: 0.1880
Epoch 4/50
140/140 [==============================] - 4s 30ms/step - loss: 2.4181 - accuracy: 0.1751 - val_loss: 2.4134 - val_accuracy: 0.1920
Epoch 5/50
140/140 [==============================] - 4s 25ms/step - loss: 2.3604 - accuracy: 0.1881 - val_loss: 2.4110 - val_accuracy: 0.1920
Epoch 6/50
140/140 [==============================] - 3s 21ms/step - loss: 2.3177 - accuracy: 0.1939 - val_loss: 2.4413 - val_accuracy: 0.1759
Epoch 7/50
140/140 [==============================] - 4s 26ms/step - loss: 2.3678 - accuracy: 0.1916 - val_loss: 2.4244 - val_accuracy: 0.18

### **Another Trial**

In [47]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.6 MB/s eta 0:00:00


In [48]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Attention
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nlpaug.augmenter.word import SynonymAug


In [49]:
# Label Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [50]:
# Parameters
embedding_dim = 64
vocab_size = len(tokenizer.word_index) + 1
num_classes = len(np.unique(y_train_encoded))  # This dynamically sets the number of classes
l2_reg = 0.01

In [51]:
# Augmentation
aug = SynonymAug(aug_src='wordnet')
def augment_text(texts, labels, augmenter, num_augmented=2):
    augmented_texts = []
    augmented_labels = []
    for text, label in zip(texts, labels):
        augmented_texts.append(text)
        augmented_labels.append(label)
        for _ in range(num_augmented):
            augmented_text = augmenter.augment(text)
            augmented_texts.append(augmented_text)
            augmented_labels.append(label)
    return augmented_texts, augmented_labels

In [52]:
X_train_aug, y_train_aug = augment_text(X_train, y_train_encoded, aug, num_augmented=1)
X_train_aug_seq = tokenizer.texts_to_sequences(X_train_aug)
X_train_aug_pad = pad_sequences(X_train_aug_seq, maxlen=100)
y_train_aug_cat = to_categorical(y_train_aug, num_classes=num_classes)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Model Definition
input_text = Input(shape=(100,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=100)(input_text)
bi_lstm = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(l2_reg)))(embedding_layer)
dropout1 = Dropout(0.5)(bi_lstm)
attention = Attention()([dropout1, dropout1])
pooling_layer = GlobalAveragePooling1D()(attention)
dropout2 = Dropout(0.5)(pooling_layer)
dense_layer = Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(dropout2)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)


In [ ]:
model = Model(inputs=input_text, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_aug_pad, y_train_aug_cat, epochs=50, batch_size=64, validation_data=(X_test_pad, y_test_cat))

Epoch 1/50
279/279 [==============================] - 47s 153ms/step - loss: 3.0514 - accuracy: 0.1707 - val_loss: 2.4931 - val_accuracy: 0.1853
Epoch 2/50
279/279 [==============================] - 26s 94ms/step - loss: 2.5062 - accuracy: 0.1719 - val_loss: 2.4447 - val_accuracy: 0.1875
Epoch 3/50
279/279 [==============================] - 18s 63ms/step - loss: 2.4857 - accuracy: 0.1733 - val_loss: 2.4093 - val_accuracy: 0.1853
Epoch 4/50
279/279 [==============================] - 12s 45ms/step - loss: 2.4603 - accuracy: 0.1732 - val_loss: 2.4250 - val_accuracy: 0.1853
Epoch 5/50
279/279 [==============================] - 12s 42ms/step - loss: 2.4578 - accuracy: 0.1777 - val_loss: 2.4465 - val_accuracy: 0.1911
Epoch 6/50
279/279 [==============================] - 9s 33ms/step - loss: 2.4748 - accuracy: 0.1808 - val_loss: 2.4754 - val_accuracy: 0.1808
Epoch 7/50
279/279 [==============================] - 7s 25ms/step - loss: 2.4833 - accuracy: 0.1790 - val_loss: 2.4675 - val_accuracy: 

In [ ]:
model.fit(X_train_aug_pad, y_train_aug_cat, epochs=100, batch_size=128, validation_data=(X_test_pad, y_test_cat))

Epoch 1/100
140/140 [==============================] - 20s 139ms/step - loss: 2.2535 - accuracy: 0.2031 - val_loss: 2.9325 - val_accuracy: 0.1655
Epoch 2/100
140/140 [==============================] - 16s 111ms/step - loss: 2.2413 - accuracy: 0.2121 - val_loss: 3.0665 - val_accuracy: 0.1624
Epoch 3/100
140/140 [==============================] - 10s 75ms/step - loss: 2.2205 - accuracy: 0.2167 - val_loss: 3.1677 - val_accuracy: 0.1606
Epoch 4/100
140/140 [==============================] - 10s 69ms/step - loss: 2.2203 - accuracy: 0.2211 - val_loss: 3.1819 - val_accuracy: 0.1525
Epoch 5/100
140/140 [==============================] - 8s 58ms/step - loss: 2.2152 - accuracy: 0.2196 - val_loss: 3.0051 - val_accuracy: 0.1575
Epoch 6/100
140/140 [==============================] - 7s 49ms/step - loss: 2.2163 - accuracy: 0.2206 - val_loss: 3.1314 - val_accuracy: 0.1651
Epoch 7/100
140/140 [==============================] - 7s 51ms/step - loss: 2.2090 - accuracy: 0.2212 - val_loss: 3.0692 - val_acc

In [ ]:
model.fit(X_train_aug_pad, y_train_aug_cat, epochs=200, batch_size=512, validation_data=(X_test_pad, y_test_cat))

Epoch 1/200
35/35 [==============================] - 10s 274ms/step - loss: 2.0010 - accuracy: 0.3070 - val_loss: 3.9611 - val_accuracy: 0.1525
Epoch 2/200
35/35 [==============================] - 6s 177ms/step - loss: 1.9936 - accuracy: 0.3077 - val_loss: 3.9491 - val_accuracy: 0.1489
Epoch 3/200
35/35 [==============================] - 7s 199ms/step - loss: 1.9839 - accuracy: 0.3122 - val_loss: 3.8818 - val_accuracy: 0.1507
Epoch 4/200
35/35 [==============================] - 5s 128ms/step - loss: 1.9784 - accuracy: 0.3141 - val_loss: 3.9934 - val_accuracy: 0.1525
Epoch 5/200
35/35 [==============================] - 5s 149ms/step - loss: 1.9694 - accuracy: 0.3162 - val_loss: 4.0735 - val_accuracy: 0.1512
Epoch 6/200
35/35 [==============================] - 6s 183ms/step - loss: 1.9601 - accuracy: 0.3234 - val_loss: 4.2488 - val_accuracy: 0.1498
Epoch 7/200
35/35 [==============================] - 5s 129ms/step - loss: 1.9542 - accuracy: 0.3229 - val_loss: 4.2350 - val_accuracy: 0.149

In [ ]:
model.fit(X_train_aug_pad, y_train_aug_cat, epochs=200, batch_size=512, validation_data=(X_test_pad, y_test_cat))

Epoch 1/200
35/35 [==============================] - 2s 59ms/step - loss: 1.6358 - accuracy: 0.4713 - val_loss: 5.8564 - val_accuracy: 0.1377
Epoch 2/200
35/35 [==============================] - 2s 44ms/step - loss: 1.6304 - accuracy: 0.4721 - val_loss: 5.9017 - val_accuracy: 0.1409
Epoch 3/200
35/35 [==============================] - 1s 31ms/step - loss: 1.6256 - accuracy: 0.4749 - val_loss: 5.8220 - val_accuracy: 0.1440
Epoch 4/200
35/35 [==============================] - 1s 32ms/step - loss: 1.6101 - accuracy: 0.4794 - val_loss: 6.1833 - val_accuracy: 0.1377
Epoch 5/200
35/35 [==============================] - 1s 31ms/step - loss: 1.6068 - accuracy: 0.4785 - val_loss: 6.0880 - val_accuracy: 0.1368
Epoch 6/200
35/35 [==============================] - 1s 38ms/step - loss: 1.6035 - accuracy: 0.4813 - val_loss: 6.0360 - val_accuracy: 0.1454
Epoch 7/200
35/35 [==============================] - 2s 44ms/step - loss: 1.5979 - accuracy: 0.4846 - val_loss: 6.2412 - val_accuracy: 0.1431
Epoch 

In [ ]:
# Checkpoint callback
checkpoint_path = "/content/drive/MyDrive/Model_data/model_checkpoint_personality_pred.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


In [ ]:
model.fit(X_train_aug_pad, y_train_aug_cat, epochs=2, batch_size=512, validation_data=(X_test_pad, y_test_cat), callbacks=[checkpoint])

Epoch 1/2
35/35 [==============================] - ETA: 0s - loss: 1.4868 - accuracy: 0.5287
Epoch 1: val_loss improved from inf to 6.62030, saving model to /content/drive/MyDrive/Model_data/model_checkpoint_personality_pred.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


35/35 [==============================] - 7s 204ms/step - loss: 1.4868 - accuracy: 0.5287 - val_loss: 6.6203 - val_accuracy: 0.1377
Epoch 2/2
35/35 [==============================] - ETA: 0s - loss: 1.4630 - accuracy: 0.5366
Epoch 2: val_loss did not improve from 6.62030
35/35 [==============================] - 1s 34ms/step - loss: 1.4630 - accuracy: 0.5366 - val_loss: 6.7086 - val_accuracy: 0.1301


In [23]:
full_dataset.head()

,TEXT,MBTI
0,httpwwwyoutubecomwatchvqsxhcwekrwhttpmediatumb...,INFJ
1,im finding lack post alarmingsex boring positi...,ENTP
2,good one _____ httpswwwyoutubecomwatchvfhigbol...,INTP
3,dear intp enjoyed conversation day esoteric ga...,INTJ
4,youre firedthats another silly misconception a...,ENTJ


1/1 [==============================] - 0s 44ms/step
The predicted MBTI personality type for the provided text is: INFP


In [53]:
model.fit(X_train_aug_pad, y_train_aug_cat, epochs=2, batch_size=64, validation_data=(X_test_pad, y_test_cat), callbacks=[checkpoint])

Epoch 1/2
279/279 [==============================] - ETA: 0s - loss: 3.0408 - accuracy: 0.1664
Epoch 1: val_loss improved from inf to 2.49538, saving model to /content/drive/MyDrive/Model_data/model_checkpoint_personality_pred.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


279/279 [==============================] - 53s 159ms/step - loss: 3.0408 - accuracy: 0.1664 - val_loss: 2.4954 - val_accuracy: 0.1853
Epoch 2/2
279/279 [==============================] - ETA: 0s - loss: 2.4971 - accuracy: 0.1751
Epoch 2: val_loss improved from 2.49538 to 2.40755, saving model to /content/drive/MyDrive/Model_data/model_checkpoint_personality_pred.h5
279/279 [==============================] - 32s 116ms/step - loss: 2.4971 - accuracy: 0.1751 - val_loss: 2.4076 - val_accuracy: 0.1920


In [60]:
from tensorflow.keras.models import load_model
model_checkpoint = "/content/drive/MyDrive/Model_data/model_checkpoint_personality_pred.h5"

model = load_model(model_checkpoint)

In [61]:


def predict_personality(text):
  preprocessed_text = preprocess_text(text)


  sequence = tokenizer.texts_to_sequences([preprocessed_text])

  padded_sequence = pad_sequences(sequence, maxlen = 100, padding='post')
  prediction = model.predict(padded_sequence)
  predicted_class_index = np.argmax(prediction)

  predicted_class = label_encoder.inverse_transform([predicted_class_index])

  return predicted_class[0]


In [62]:
sample_text = "Radom people suck. Happy birthday to you. Good luck to you and have a great day"
predicted_mbti = predict_personality(sample_text)
print(f"The predicted MBTI personality type for the provided text is: {predicted_mbti}")

1/1 [==============================] - 1s 717ms/step
The predicted MBTI personality type for the provided text is: INFP
